# <center >谷歌开源Agent框架ADK开发实战</center>
## <center>Part 5. ADK 构建Agent及自定义外部工具最佳实践</center>

&emsp;&emsp;在`Google ADK`框架的前几节课程内容，我们主要给大家讲解的是`Google ADK`整个框架底层的运行原理，涉及的是`Runner`、`Agent`和`SessionService`三个核心组件的内置处理机制及整体的数据流向。实际上，无论我们基于`Google ADK`开发的应用多么复杂，其底层都同样遵循着这一套设计模式，因此先理解这种设计模式，对我们在设计和构建上层应用时是非常有帮助的。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202506051536794.png" width=60%></div>

&emsp;&emsp;而从本节课开始，我们就将开始逐步的进入到`Google ADK`的实战开发阶段。

&emsp;&emsp;首先我们先了解一下在`ADK`框架中都可以构建哪些类型的`Agent`，各个类型`Agent`的特点和适用的场景是有很大区别的。但在此之前，需要重点的给大家介绍一下`ADK`中的`State`临时状态，因为它在实际需求开发场景中，在智能体构建过程中会起到非常关键的作用。

# 一、State 临时状态应用

&emsp;&emsp;每个会话 (`Session`) 都维护一个 `state` 对象，用于存储对当前对话有用的动态信息。可以把它理解为智能体运行过程中的“草稿板”或短期记忆，用于记录用户提供的偏好、任务进度、临时结果等数据。`State` 是一个键值对集合，它的特点是只在当前会话范围内有效，随着对话进展不断更新。其源码如下：https://github.com/google/adk-python/blob/main/src/google/adk/sessions/state.py

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202505261514002.png" width=60%></div>

&emsp;&emsp;`State`虽然是作为临时状态，但也是作用于`Session`对象中，因此`State`的更新和存储也是依托于`Session`对象，同时采用前缀来区分不同的作用域，即：

- **无前缀**：特定于会话，仅在当前会话中持续存在；
- **app:** 应用级状态，应用程序范围，在所有用户和会话之间共享；
- **user:** ，用户级状态，在特定用户的所有会话中持续存；
- **temp:** ：处理临时状态，仅在当前执行周期内存在；

&emsp;&emsp;这里能够理解的是，`State`其实是`Session`的一部分，可以在运行时灵活的指定一些数据临时存储在`State`中。而在存储时，则遵循`SessionService`的实现。不同的`SessionService`实现，则`state`的存储形式也会不同。如果是`InMemorySessionService`，则`State`只存储在内存中，当程序重启时会丢失；如果是`DatabaseSessionService`，则`State`会存储到数据库中，当程序重启时会继续存在。存储的基本原则仍然是按照`app`、`user`、`session`三个层级来存储，而`temp`中存储的数据，不论是`InMemorySessionService`还是`DatabaseSessionService`，都会在运行结束后被清空。

&emsp;&emsp;`state`作为一种键值对的存储结构，会随着业务场景的复杂度呈现出不一样的状态。当你的业务场景越复杂，涉及到需要处理`state` 数据也会越多。而像简单的问答场景，甚至并不需要我们对`state`做任何处理。因此在`ADK`的设计中，`state` 也有着不同的应用方法，这里我们会主要区分成以下三种：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>`ADK三种状态管理方式</font></p>
<div class="center">

| 方法 | 执行时机 | 控制粒度 | 适用场景 |
|------|----------|----------|----------|
| **`output_key`** | Agent 响应完成后 | 粗粒度 | 简单的响应保存 |
| **`state_delta`** | 任意时刻 | 最细粒度 | 复杂逻辑、系统事件 |
| **`ContextContext/ToolContext`** | 回调/工具执行期间 | 中等粒度 | 执行过程中的状态管理 |

&emsp;&emsp;接下来我们需要依次结合实际的应用场景示例，帮助大家理解三种不同状态管理方式的最佳应用实践。

## 1.1 state 默认处理机制

&emsp;&emsp;这里我们可以逐步看一下`Google ADK`在运行时`state`的变化情况。为了方便进行测试，我们这里使用预构建的`app`、`user`、`session`，并使用`DatabaseSessionService`进行测试。

In [1]:
import os
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm
from google.adk.runners import Runner
from google.adk.agents.run_config import RunConfig, StreamingMode
from google.genai import types 
from dotenv import load_dotenv
load_dotenv(override=True)

# 配置 模型信息 与 Session 信息
DS_API_KEY = os.getenv("DS_API_KEY")
DS_BASE_URL = os.getenv("DS_BASE_URL")

# 预构建 应用、用户、会话
APP_NAME = "test_app"
USER_ID = "user_101"
SESSION_ID = "session_101"  

# 创建模型和 Agent
model = LiteLlm(
    model="deepseek/deepseek-chat",  
    api_base=DS_BASE_URL,
    api_key=DS_API_KEY
)

In [2]:
# 数据库连接配置
DB_CONFIG = {
    'host': 'localhost',  # 这里替换成实际的 PostgreSQL 服务器地址
    'port': 5432,   # 这里替换成实际的 PostgreSQL 服务器端口
    'database': 'adk',  # 这里替换成实际的 PostgreSQL 数据库名称
    'user': 'postgres',  # 这里替换成实际的 PostgreSQL 用户名
    'password': 'snowball2019'  # 这里替换成实际的 PostgreSQL 密码
}

&emsp;&emsp;接下来我们配置数据库连接信息，并创建`SessionService`对象：

In [3]:
from google.adk.sessions import DatabaseSessionService

# 数据库配置
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'adk',
    'user': 'postgres',
    'password': 'snowball2019'
}

# 生成数据库连接字符串
DATABASE_URL = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

# 创建 SessionService 对象
session_service = DatabaseSessionService(DATABASE_URL)

&emsp;&emsp;构造 `Agent`对象实例：

In [4]:
agent = Agent(
    name="chatbot",
    model=model,
    instruction="你是一个乐于助人的中文助手。请根据用户的问题给出回答。",
)

&emsp;&emsp;创建 `Runner`对象实例：

In [5]:
# 创建 Runner
runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service,
)

&emsp;&emsp;使用`create_session`方法在`PostgreSQL`中生成`Session`数据。

In [6]:
# 创建会话 Session
test_session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
    )

print(f"✅ 会话创建成功: {test_session.id}")

✅ 会话创建成功: session_101


&emsp;&emsp;获取刚刚创建的`Session`对象，查看在运行前`Session`对象的`state`存储的数据情况：

In [7]:
print("=== ADK 默认状态结构演示 ===\n")

# 2. 获取 刚刚创建的 Session 对象
session = await session_service.get_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
    )

print("1. 初始会话的默认状态:")
print(f"   session.state 类型: {type(session.state)}")
print(f"   session.state 内容: {session.state}")
print(f"   session.events 长度: {len(session.events)}")

=== ADK 默认状态结构演示 ===

1. 初始会话的默认状态:
   session.state 类型: <class 'dict'>
   session.state 内容: {}
   session.events 长度: 0


&emsp;&emsp;通过输出结果可以看到，当执行`create_session`方法进行初始化操作时，`Session`对象的`state`是一个空字典，同时也不触发任何`events`事件。接下来我们实际运行`Runner`, 再次打印`Session`对象的`state`和`events`信息以观察`state`临时状态中的数据变化情况。

In [8]:
# 3. 运行一个简单对话并查看Events中的状态
print("2. 运行对话前的状态:")
query = "你好，请你介绍一下你自己。"

# 将用户的问题转换为 ADK 格式
content = types.Content(role='user', parts=[types.Part(text=query)])

# 异步运行
async for event in runner.run_async(
    user_id=USER_ID,
    session_id=SESSION_ID,
    new_message=content,
):
    # 打印事件信息
    print(f"   - Author: {event.author}")
    print(f"   - Content type: {type(event.content)}")
    print(f"   - Actions: {event.actions}")

    # 检查事件中的状态相关信息
    if hasattr(event.actions, 'state_delta') and event.actions.state_delta:
        print(f"   - State delta: {event.actions.state_delta}")
    else:
        print(f"   - State delta: None (默认为空)")
        
    # 打印最终响应
    if event.content and event.content.parts:
        print("🤖", event.content.parts[0].text)

2. 运行对话前的状态:
   - Author: chatbot
   - Content type: <class 'google.genai.types.Content'>
   - Actions: skip_summarization=None state_delta={} artifact_delta={} transfer_to_agent=None escalate=None requested_auth_configs={}
   - State delta: None (默认为空)
🤖 你好！我是一个人工智能助手，旨在为你提供各种信息和帮助。我可以回答你的问题、提供建议、协助解决问题，或者陪你聊天。无论是学习、工作还是日常生活相关的话题，我都会尽力为你提供有用的信息。  

我的知识涵盖多个领域（如科技、历史、文化、健康等），并能处理多种任务（如翻译、计算、写作辅助等）。我没有情感或自我意识，但会尽量以友好、专业的方式与你交流。  

如果你有任何问题或需要帮助，随时告诉我！ 😊


&emsp;&emsp;当运行结束后，再次获取`Session`对象的`state`和`events`信息，`state`中仍然是空字典的初始状态，而`events`中则存储了完整的对话历史记录。（大家也可以在`PostgreSQL`中验证）

In [9]:
print("\n3. 对话后的会话状态:")
updated_session = await session_service.get_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
    )

print(f"   updated_session.state 内容: {updated_session.state}")
print(f"   updated_session.events 长度: {len(updated_session.events)}")

for index,event in enumerate(updated_session.events):
    print(f"   events: {index} - {event}")



3. 对话后的会话状态:
   updated_session.state 内容: {}
   updated_session.events 长度: 2
   events: 0 - content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='你好，请你介绍一下你自己。')], role='user') grounding_metadata=None partial=None turn_complete=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=None invocation_id='e-40a4f103-590c-403a-817c-7be3b851b3cf' author='user' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}) long_running_tool_ids=set() branch=None id='Dkc9GlW4' timestamp=1749617647.205704
   events: 1 - content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, fu

&emsp;&emsp;通过这个手动构建的流程，在`ADK`中的`SessionService`、`Runner`及`Agent`中，其实都不依赖自定义`State`，默认空字典{}照样正常运行，只有事件是随着对话的进行而不断追加的。这也就是为什么我们在之前课程的所有案例中，没有了解`state`这个概念但是仍然可以正常运行的原因。

&emsp;&emsp;`State`真正的作用是供开发者提供自定义流程的入口，其核心是用来扩展`ADK`的灵活性和复杂性的。对于一些基础的流程，如简单问答："今天天气怎么样？"、代码生成："写个排序函数"、一次性翻译、摘要等，其实都不需要自定义`State`。但是对一些复杂流程，如多步骤任务：订票流程、上下文对话："继续刚才的话题"、Agent协作：数据传递等，则还需要自定义`State`来存储一些临时数据从而支撑数据的传递，大家可以理解为：

- 📊 State (状态) - 结构化的键值对数据
   - 用途: 存储配置、偏好、设置、当前状态
   - 特点: 可读写、可更新、结构化
   - 示例: 用户语言偏好、任务进度、多代理协作间的信息传递

- 📝 Event (事件) - 时序化的历史记录
   - 用途: 记录对话历史、操作记录、状态变更
   - 特点: 只写入、不可修改、时间序列
   - 示例: 用户消息、AI回复、工具调用、错误记录


&emsp;&emsp;理解了`state`的概念及在`ADK`中的默认处理流程后，我们可以首先针对`state_delta`这种手动处理状态数据的方式来了解下：`state`在特定复杂应用场景中的使用方法和技巧。

## 1.1 state_delta 复杂场景应用

&emsp;&emsp;`state` 的触发时机是：每当产生新的交互，先通过 `SessionService` 将对应的 `Event` 追加到会话历史 (`session.events`) 中，并据此更新会话的 `State`。更深入细节，则是`SessionService` 会读取 `Event` 中的状态变更指令（`state_delta`），合并到会话的 `state` 中，并处理持久化（例如`InMemorySessionService`或`DatabaseSessionService`）。一句话总结：`Event` 是`state`的载体，`state`是`Event`的存储结果。所以才有我们看到的默认情况下`Event`中的`state`是：

- 新`Session`的默认状态：`session.state` 返回空字典 `{}`
- `Event`的默认`state_delta`：`event.actions.state_delta` 默认为 `None`

- `State`对象内部结构：
  - _value: {} (持久化状态，初始为空)
  - _delta: {} (待提交变更，初始为空)

&emsp;&emsp;举一个实际的示例：当用户登录时，我们希望更新多个状态值（如登录次数、最后登录时间等），这可以通过构造一个带有 `state_delta` 的 `Event` 来实现：

In [10]:
APP_NAME = "test_app"
USER_ID = "user_101"
SESSION_ID = "session_102" # 这里新建一个 102 的会话

# 创建 Runner
runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service,
)

# 测试创建会话
login_session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
    state={"user:login_count": 0, "task_status": "idle"}  # 在创建会话时，指定初始状态,idle表示空闲状态
    )

In [11]:
print("1. 初始会话的默认状态:")
print(f"   login_session.state 类型: {type(login_session.state)}")
print(f"   login_session.state 内容: {login_session.state}")
print(f"   login_session.events 长度: {len(login_session.events)}")

1. 初始会话的默认状态:
   login_session.state 类型: <class 'dict'>
   login_session.state 内容: {'task_status': 'idle', 'user:login_count': 0}
   login_session.events 长度: 0


&emsp;&emsp;可以看到，在创建会话时，因为我们指定了初始状态`state={"user:login_count": 0, "task_status": "idle"}`，所以`Session`对象的`state`中会包含这两个字段。接下来，其实我们就可以围绕这两个字段来构造`Event`，并更新`Session`的`state`。


&emsp;&emsp;首先构造一个系统登录成功事件，并写入`state`中进行事件状态传递：

In [12]:
import time
from google.adk.events import Event, EventActions

def login_success_delta(sess):
    ts = time.time()
    return {
        "task_status": "active",   # 无前缀，则作用于会话范围
        "user:login_count": sess.state.get("user:login_count", 0) + 1, # user 前缀，则作用于用户范围
        "user:last_login_ts": ts, #user 前缀，则作用于用户范围
        "temp:validation_needed": True        # temp 前缀，临时状态，仅本轮有效
    }

current_time = time.time()

# 生成状态变更
state_changes = login_success_delta(sess=login_session)  # 这里要替换为login_session

# 生成事件
actions = EventActions(state_delta=state_changes)
system_event = Event(
    invocation_id="login_update",
    author="system",
    actions=actions,
    timestamp=current_time
)

await session_service.append_event(session=login_session, event=system_event)  # 这里要替换为login_session
print("\n✅ 已写入登录增量")


✅ 已写入登录增量（含 temp:validation_needed）


&emsp;&emsp;然后再次获取`Session`对象的`state`和`events`信息：

In [13]:
# 2. 获取 刚刚创建的 Session 对象
update_session = await session_service.get_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
    )

print("2. 系统事件写入后的状态:")
print(f"   update_session.state 类型: {type(update_session.state)}")
print(f"   update_session.state 内容: {update_session.state}")
print(f"   update_session.events 长度: {len(update_session.events)}")
print(f"   update_session.events 内容: {update_session.events[0].invocation_id}")

2. 系统事件写入后的状态:
   update_session.state 类型: <class 'dict'>
   update_session.state 内容: {'task_status': 'active', 'user:login_count': 1, 'user:last_login_ts': 1749618387.5234833}
   update_session.events 长度: 1
   update_session.events 内容: login_update


&emsp;&emsp;可以看到，`Session`对象的`state`中已经包含`task_status`和`user:login_count`字段，并且`user:last_login_ts`字段也被更新了。注意：这里为什么读不到`temp:validation_needed`字段？ 是因为带`temp: xxx`前缀的键会在一次提交后立即丢弃，所以读不到它。如何验证临时键只存在一次？ 最简单的方法就是完成事件写入后，通过`state_delta`查看，如下代码所示：

In [14]:
# 测试创建会话
test_temp_session = await session_service.create_session(
    app_name="test_app",
    user_id="user_101",
    session_id="session_103",  # 新创建一个 session 会话
    state={"user:login_count": 0, "task_status": "idle"}  # 在创建会话时，指定初始状态,idle表示空闲状态
    )

In [15]:
current_time = time.time()
state_changes = login_success_delta(sess=test_temp_session)  # 这里要替换为test_temp_session

actions = EventActions(state_delta=state_changes)

system_event = Event(
    invocation_id="login_update",
    author="system",
    actions=actions,
    timestamp=current_time
)

await session_service.append_event(session=test_temp_session, event=system_event)  # 这里要替换为test_temp_session
print("\n✅ 已写入登录增量（含 temp:validation_needed）")


✅ 已写入登录增量（含 temp:validation_needed）


&emsp;&emsp;`temp`前缀的设计目标就是 “只在本轮调用链里用一下，然后马上销毁”，在 `DatabaseSessionService.append_event()` 里，合并逻辑会先把 `temp:` 前缀过滤掉，再更新 `session.state`，所以打印是看不到它的，比如如下所示：

In [16]:
print("测试 temp: 前缀状态是否存在")
print(f"   test_temp_session.state 类型: {type(test_temp_session.state)}")
print(f"   test_temp_session.state 内容: {test_temp_session.state}")
print(f"   test_temp_session.events 长度: {len(test_temp_session.events)}")

测试 temp: 前缀状态是否存在
   test_temp_session.state 类型: <class 'dict'>
   test_temp_session.state 内容: {'task_status': 'active', 'user:login_count': 1, 'user:last_login_ts': 1749618473.5107284}
   test_temp_session.events 长度: 1


&emsp;&emsp;能够看到哪怕不刷新`Session`对象，`temp:validation_needed`字段也已经不存在了。因为其内部调用链顺序其实是这样的：

```json
    append_event(...)
    └─ ① 事件写入 events 表
    └─ ② 取 actions.state_delta
        ├─ 删除所有以 "temp:" 开头的键
        └─ 把剩余键合并进 session.state   ← 打印时看到的内容
```

&emsp;&emsp;因此，如果真正想看临时键，则需要直接通过事件里的 `state_delta` 来获取：

In [17]:
print(system_event.actions.state_delta)

{'task_status': 'active', 'user:login_count': 1, 'user:last_login_ts': 1749618473.5107284, 'temp:validation_needed': True}


&emsp;&emsp;`temp`前缀的键主要用来在同一轮内部让多个工具/子Agent传递短期标志数据（比如“当前回合已做过身份验证”），有效防止把纯瞬时数据写进持久存储，避免污染会话状态或占用数据库容量。这一点，我们会在接下来的课程中讲解使用工具/MCP构建复杂`Multi-Agent`时，再详细展开讲解，这里大家先做了解。

&emsp;&emsp;现在，我们再回到原始流程中。现在我们已经通过`Event`的`state_delta`实现了对`Session`的`state`的更新，那么`state`中的数据就可以用来做逻辑的处理从而影响`Agent`的运行状态，比如下面这个场景示例： 我们通过`Session`的`state`中的字段来模拟用 `ADK` 在后台做“登录 + 首次验证”流程，并用同一个持久会话继续对话。

&emsp;&emsp;首先新建一个`Session`对象，并写入初始状态：

In [18]:
import os
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm
from google.adk.runners import Runner
from google.genai import types 
from google.adk.sessions import DatabaseSessionService
from dotenv import load_dotenv
load_dotenv(override=True)

# 配置 模型信息 与 Session 信息
DS_API_KEY = os.getenv("DS_API_KEY")
DS_BASE_URL = os.getenv("DS_BASE_URL")

# 配置 应用信息 与 用户信息
APP_NAME = "test_app"
USER_ID = "user_101"
SESSION_ID = "session_110" # 这里新建一个的会话

# 创建模型和 Agent
model = LiteLlm(
    model="deepseek/deepseek-chat",  
    api_base=DS_BASE_URL,
    api_key=DS_API_KEY
)

# 数据库配置
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'adk',
    'user': 'postgres',
    'password': 'snowball2019'
}

# 生成数据库连接字符串
DATABASE_URL = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

# 创建 SessionService 对象
session_service = DatabaseSessionService(DATABASE_URL)


# 创建 Agent对象
agent = Agent(
    name="chatbot",
    model=model,
    instruction="你是一个乐于助人的中文助手。请用中文回答用户的问题",
)

# 创建 Runner
runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service,
)

# 测试创建会话
login_session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
    state={"user:login_count": 0, "task_status": "idle"}  # 在创建会话时，指定初始状态,idle表示空闲状态
    )

&emsp;&emsp;然后，我们再写入一个系统初始化事件，探测网页状态，即如果有人访问的话，则进行初始化，并更新`Session`的`state`：

In [22]:
import time
from google.adk.events import Event, EventActions

# ── 系统后台：写入一次登录事件 --------------------------
async def mark_login_success():
    # 先获取 Session 对象
    sess = await session_service.get_session(
            app_name=APP_NAME,
            user_id=USER_ID,
            session_id=SESSION_ID,
            )

    # 构造状态变更
    delta = {
        "task_status": "active",
        "user:login_count": sess.state.get("user:login_count", 0) + 1,
        "validation_needed": True  # 这里表示需要验证
    }

    # 写入事件
    await session_service.append_event(
        session=sess,
        event=Event(author="chatbot",
                    invocation_id="login_update",
                    actions=EventActions(state_delta=delta),
                    timestamp=time.time())
    )
    print("✅ 已写入登录增量（validation_needed=True）")

&emsp;&emsp;接下来，我们再写一个网关函数，检查是否需要验证用户身份信息：

In [23]:
from google.adk.events import Event, EventActions
from google.genai import types 

# ── 网关函数：检查是否需要验证 ------------------------------------------
async def maybe_run_agent(user_input: str, code: str | None = None):
    # 先获取 Session 对象
    sess = await  session_service.get_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

    # 获取需要验证的标志
    need_check = sess.state.get("validation_needed", False)

    # 需要验证但尚未通过
    if need_check:
        if code == "123456":  # 这里假设是验证码场景
            # 写事件把 validation_needed 置 False
            await session_service.append_event(
                session=sess,
                event=Event(
                    author="chatbot",
                    invocation_id="validate_ok",
                    actions=EventActions(state_delta={"validation_needed": False}),
                    timestamp=time.time()
                )
            )
            # 重新拉一次最新会话，防止读到旧缓存
            sess = await session_service.get_session(
                app_name=APP_NAME,
                user_id=USER_ID, 
                session_id=SESSION_ID
            )
            print("🔓 验证成功，放行对话")
        else:
            print("⚠️  需要先完成账号验证。请提交验证码。")
            return

    content = types.Content(role='user', parts=[types.Part(text=user_input)])
    
    # 如果已登录，正常放行，调用Agent生成回复
    async for ev in runner.run_async(
        user_id=USER_ID,
        session_id=SESSION_ID,
        new_message=content,
    ):
        # 打印回复
        if ev.content and ev.content.parts:
            print("🤖", ev.content.parts[0].text)

&emsp;&emsp;下面进行业务流程的测试。首先，我们进行初始化（对应的业务场景其实就是用户打开某一个页面）：

In [24]:
await mark_login_success()

✅ 已写入登录增量（validation_needed=True）


&emsp;&emsp;接下来，尝试未登录状态下发起会话（对应的业务场景就是用户打开页面后，直接在对话框进行提问，而事先没有进行登录）：

In [25]:
await maybe_run_agent("你好，很高兴认识你~")   # 不带验证码 → 会被拦截

⚠️  需要先完成账号验证。请提交验证码。


&emsp;&emsp;然后，模拟登录成功的情况，即携带验证码发起会话（对应的业务场景是系统检测到用户没有登录，弹出一个登录框要求用户进行登录）：

In [26]:
await maybe_run_agent(
    user_input="你好，请你介绍一下你自己", # 用户输入
    code="123456"  # 提交验证码
    ) 

🔓 验证成功，放行对话
🤖 你好！我是一个智能助手，旨在为你提供各种帮助和解答问题。我可以协助你完成以下任务：

1. **信息查询**：提供知识解答、实时资讯（如天气、新闻等）。  
2. **实用工具**：翻译、计算、单位换算、时间管理等。  
3. **创意生成**：写作灵感、文案建议、故事创作等。  
4. **学习辅导**：解释概念、解题思路、语言学习等。  
5. **日常建议**：旅行规划、健康小贴士、娱乐推荐等。  

我没有情感或主观意识，但会尽量用友好易懂的方式与你互动。如果有任何需求，随时告诉我哦！ 😊  

你想先从哪方面开始呢？


&emsp;&emsp;最后，登录成功后，即可不再重复携带验证码，直接进行对话：

In [27]:
await maybe_run_agent(user_input="介绍一下什么是大模型？")

🤖 大模型（**Large Language Model, LLM**）是指基于海量数据和庞大参数规模训练的人工智能模型，能够处理自然语言理解、生成、推理等复杂任务。以下是其核心特点和工作原理的简介：

---

### **1. 核心特点**
- **参数规模大**：参数量通常达到数十亿甚至万亿级别（例如GPT-3有1750亿参数），赋予模型更强的学习能力。
- **多任务通用性**：可处理翻译、问答、写作、编程等多种任务，无需针对每项任务单独训练。
- **上下文学习**：通过分析输入文本的上下文生成连贯、逻辑合理的输出。

---

### **2. 关键技术**
- **Transformer架构**：基于自注意力机制（Self-Attention），能高效捕捉长距离文本依赖关系。
- **预训练+微调**：
  - **预训练**：在超大规模文本数据（如书籍、网页）上无监督学习，掌握语言统计规律。
  - **微调**：针对特定任务（如客服、医疗）用标注数据进一步优化。
- **提示工程（Prompting）**：通过设计输入提示（Prompt）引导模型生成所需输出。

---

### **3. 典型应用场景**
- **生成类**：自动写作、代码生成、对话机器人（如ChatGPT）。
- **分析类**：文本分类、情感分析、信息抽取。
- **交互类**：智能客服、虚拟助手、教育辅导。

---

### **4. 局限性**
- **事实性错误**：可能生成看似合理但实际错误的内容（称为“幻觉”）。
- **数据偏见**：训练数据中的偏见可能反映在输出中。
- **资源消耗**：训练和部署需大量算力，成本高昂。

---

### **5. 代表模型**
- **GPT系列**（OpenAI）：生成能力突出，迭代至GPT-4。
- **BERT**（Google）：擅长理解任务，如搜索、分类。
- **PaLM**（Google）、**LLaMA**（Meta）：开源或高效能替代方案。

---

如果需要更具体的细节（如技术原理、应用案例等），可以进一步探讨！ 😊


&emsp;&emsp;这个流程其实就是一套比较标准的在`ADK`框架中借助`state`来快速实现用户登录校验的实现机制，其实并不需要我们手动去写额外的处理逻辑，仅仅需要灵活应用`state`的处理机制。同时我们也可以在数据库中看到实时的数据持久化存储信息，如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202506051432018.png" width=80%></div>

&emsp;&emsp;由此大家可以理解到的是：追加 `Event` 是更新 `State` 的机制。`SessionService` 是通过读取 `Event` 中的状态变更指令（`state_delta`），合并到会话的 `state` 中并进行处理的。所以可以把`State`理解为短期记忆，用于存储当前对话的上下文和临时信息，在上下文中是可变的，随着对话过程会不断更新。而`Event` 是长期记忆，用于存储执行过程某个具体步骤的结果，如用户输入、工具调用、错误信息等， 是不可变的，一旦生成后不会改变。其流程我们可以梳理如下：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202506101454990.png" width=80%></div>


&emsp;&emsp;上述流程属于手动更新 `State` 的方法。适用于复杂的场景，比如需要在`Agent Workflow`中维护多个键，需要明确特定范围（如 `user:` 或 `app:` ）等情况，通过在 `EventActions` 中手动构建 `state_delta` 从而实现定制化的处理。实际上，工业开发场景中使用这种手动更新的方式是需要频繁且灵活应用的一种开发形式，比如上面案例实现的用户登录权限校验、跨`Agent`协作等复杂工作流设计等。大家务必理解并掌握`state`的手动更新方法。

## 1.2 output_key 简单场景应用

&emsp;&emsp;除了手动更新`State` 的方法，`Google ADK`框架在处理`State` 时，还提供了一个更加简单的自动更新方法，即可以直接在构建`Agent`的时候，通过`output_key` 参数指定`State` 的键名，从而实现自动更新。其源码的定义是这样的：https://github.com/google/adk-python/blob/main/src/google/adk/agents/llm_agent.py

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202506101518185.png" width=80%></div>

&emsp;&emsp;`out_key`的核心处理逻辑在`__maybe_save_output_to_state`函数中，其核心逻辑如下：

```python
    def __maybe_save_output_to_state(self, event: Event):
      """Saves the model output to state if needed."""
      if (
          self.output_key                 # 1. 设置了 output_key
          and event.is_final_response()  # 2. 是最终响应事件
          and event.content              # 3. 事件有内容
          and event.content.parts        # 4. 内容有文本部分
      ): 
        result = ''.join(
            [part.text if part.text else '' for part in event.content.parts]
        )
        if self.output_schema:
          result = self.output_schema.model_validate_json(result).model_dump(
              exclude_none=True
          )
        event.actions.state_delta[self.output_key] = result
```

&emsp;&emsp;四个触发条件必须同时满足才会进行`State`的更新，对应的底层实现的流程是`Runner`会使用 `output_key` 和 `state_delta` 创建必要的 `EventActions` 并调用 `append_event` 。其核心逻辑如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202506101526418.png" width=80%></div>

&emsp;&emsp;这也解释了为什么 `output_key` 只适合简单场景：因为它只能在 `Agent` 完成响应后被动保存文本内容，无法处理复杂的状态逻辑或主动控制更新时机。比如我们刚才实现的登录验证场景，依次需要：

- 需求1：需要系统后台操作：登录时写入状态，更新 `user:login_count`，这不是 `Agent` 响应；
- 需求2：需要条件控制：根据状态决定是否允许 `Agent` 运行， 这不是 `Agent` 响应；
- 需求3：需要多步骤状态管理：登录→验证→清除标志，验证成功时清除 `validation_needed`，这不是 `Agent` 响应；

&emsp;&emsp;这些需求无法通过 `output_key` 的被动更新机制来实现，需要手动控制状态更新时机。这也就是为什么说企业应用开发中更加常用的是手动更新 `State` 的方法。这一点需要大家理解，`output_key`和`state_delta` 是两种不同的状态管理方式在应用时的本质区别。

&emsp;&emsp;我们这里可以实现一个适合`output_key`的场景，比如在聊天机器人中记录用户的满意度。如下代码所示：

In [45]:
import os
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm
from google.adk.runners import Runner
from google.genai import types 
from google.adk.sessions import DatabaseSessionService
from dotenv import load_dotenv
load_dotenv(override=True)

# 配置 模型信息 与 Session 信息
DS_API_KEY = os.getenv("DS_API_KEY")
DS_BASE_URL = os.getenv("DS_BASE_URL")

# 配置 应用信息 与 用户信息
APP_NAME = "test_app_2"
USER_ID = "user_102"
SESSION_ID = "session_123" # 这里新建一个的会话

# 创建模型和 Agent
model = LiteLlm(
    model="deepseek/deepseek-chat",  
    api_base=DS_BASE_URL,
    api_key=DS_API_KEY
)

# 数据库配置
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'adk',
    'user': 'postgres',
    'password': 'snowball2019'
}

# 生成数据库连接字符串
DATABASE_URL = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

# 创建 SessionService 对象
simple_session_service = DatabaseSessionService(DATABASE_URL)

&emsp;&emsp;我们现在可以通过`output_key`指定一个`last_response`，用来在`state`字段中自动保存模型的最终回复，如下所示：

In [46]:
# 场景1：普通聊天机器人 - 保存最后回复
chat_agent = Agent(
    name="ChatBot",
    model=model,
    instruction="你是一个乐于助人的智能小助手，用中文简短回答",
    output_key="last_response"  # 自动保存响应到 state["last_response"]
)

In [47]:
# 创建会话
simple_session = await simple_session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
)

print(f"初始状态: {simple_session.state}")

初始状态: {}


In [48]:
# === 场景1：普通对话 ===
chat_runner = Runner(
    agent=chat_agent,
    app_name=APP_NAME,
    session_service=simple_session_service
)

print("\n=== 场景1：普通对话（保存最后回复）===")
user_msg = types.Content(role='user', parts=[types.Part(text="你好，请你简短的介绍一下你自己~")])

# 运行
async for event in chat_runner.run_async(
    user_id=USER_ID,
    session_id=SESSION_ID,
    new_message=user_msg
):
    if event.content and event.content.parts:
        print(f"🤖 回复: {event.content.parts[0].text}")

# 检查状态更新
updated_session = await simple_session_service.get_session(
    app_name=APP_NAME, 
    user_id=USER_ID, 
    session_id=SESSION_ID
)
print(f"状态更新: last_response = {updated_session.state.get('last_response', 'None')}")


=== 场景1：普通对话（保存最后回复）===
🤖 回复: 你好！我是ChatBot，一个智能助手，随时为你提供简短、高效的帮助。无论是解答问题、提供建议还是日常闲聊，我都会尽力协助你！有什么需要随时告诉我~ 😊
状态更新: last_response = 你好！我是ChatBot，一个智能助手，随时为你提供简短、高效的帮助。无论是解答问题、提供建议还是日常闲聊，我都会尽力协助你！有什么需要随时告诉我~ 😊


&emsp;&emsp;

&emsp;&emsp;接下来我们再使用`output_key` 来实现满意度评分场景，如下所示：

In [49]:
# 场景2：满意度评分机器人 - 保存评分
rating_agent = Agent(
    name="ChatBot", 
    model=model,
    instruction="""
    根据用户反馈给出1-5分的满意度评分，只输出数字。
    1分=非常不满意，5分=非常满意
    """,
    output_key="user:satisfaction_score"  # 保存到用户级状态
)

&emsp;&emsp;这里我们复用之前的`Session` 对象，来实现满意度评分场景，如下所示：

In [52]:
# 创建会话
simple_session = await simple_session_service.get_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
)

print(f"初始状态: {simple_session.state}")

# === 场景2：满意度评分 ===
rating_runner = Runner(
    agent=rating_agent,
    app_name=APP_NAME,
    session_service=simple_session_service
)

print("\n=== 场景2：满意度评分（保存到用户状态）===")
feedback_msg = types.Content(role='user', parts=[types.Part(text="回答很详细，我很满意")])

async for event in rating_runner.run_async(
    user_id=USER_ID,
    session_id=SESSION_ID,
    new_message=feedback_msg
):
    if event.content and event.content.parts:
        print(f"📊 评分: {event.content.parts[0].text}")

# 检查用户级状态
updated_session = await simple_session_service.get_session(
    app_name=APP_NAME, 
    user_id=USER_ID, 
    session_id=SESSION_ID
)

print(f"状态更新: user:satisfaction_score = {updated_session.state.get('user:satisfaction_score', 'None')}")

初始状态: {'last_response': '你好！我是ChatBot，一个智能助手，随时为你提供简短、高效的帮助。无论是解答问题、提供建议还是日常闲聊，我都会尽力协助你！有什么需要随时告诉我~ 😊', 'user:satisfaction_score': '5'}

=== 场景2：满意度评分（保存到用户状态）===
📊 评分: 5
状态更新: user:satisfaction_score = 5


&emsp;&emsp;接下来再实现一个会话总结场景，如下所示：

In [53]:
# 创建会话
simple_session = await simple_session_service.get_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
)

print(f"初始状态: {simple_session.state}")

# 场景3：总结机器人 - 保存会话总结
summary_agent = Agent(
    name="ChatBot",
    model=model, 
    instruction="用一句话总结对话要点",
    output_key="session_summary"  # 保存会话总结
)


# === 场景3：会话总结 ===
summary_runner = Runner(
    agent=summary_agent,
    app_name=APP_NAME,
    session_service=simple_session_service
)

print("\n=== 场景3：会话总结 ===")
summary_msg = types.Content(role='user', parts=[types.Part(text="总结一下我们的对话")])

async for event in summary_runner.run_async(
    user_id=USER_ID,
    session_id=SESSION_ID,
    new_message=summary_msg
):
    if event.content and event.content.parts:
        print(f"📝 总结: {event.content.parts[0].text}")

# 最终状态
final_session = await simple_session_service.get_session(
    app_name=APP_NAME, 
    user_id=USER_ID, 
    session_id=SESSION_ID
)

print(f"\n🎯 最终状态: {final_session.state}")

初始状态: {'last_response': '你好！我是ChatBot，一个智能助手，随时为你提供简短、高效的帮助。无论是解答问题、提供建议还是日常闲聊，我都会尽力协助你！有什么需要随时告诉我~ 😊', 'user:satisfaction_score': '5'}

=== 场景3：会话总结 ===
📝 总结: 1. **自我介绍**：我简短介绍了自己作为智能助手的角色和功能。  
2. **反馈互动**：你表示满意后，我用简洁的“5”回应，之后再次确认总结。  
3. **最终总结**：用一句话概括对话流程：从介绍到正向反馈的极简互动。  

（注：若需更短版本：“用户满意我的简短介绍，互动愉快。” 😄）

🎯 最终状态: {'last_response': '你好！我是ChatBot，一个智能助手，随时为你提供简短、高效的帮助。无论是解答问题、提供建议还是日常闲聊，我都会尽力协助你！有什么需要随时告诉我~ 😊', 'session_summary': '1. **自我介绍**：我简短介绍了自己作为智能助手的角色和功能。  \n2. **反馈互动**：你表示满意后，我用简洁的“5”回应，之后再次确认总结。  \n3. **最终总结**：用一句话概括对话流程：从介绍到正向反馈的极简互动。  \n\n（注：若需更短版本：“用户满意我的简短介绍，互动愉快。” 😄）', 'user:satisfaction_score': '5'}


&emsp;&emsp;综上，`output_key` 虽然使用起来比较简单，但是也仅仅适用于这种简单的键值存储，且不需要复杂逻辑的被动更新场景。因此，强烈建议大家在实际的开发过程中更多的应用手动更新 `State` 的方法以适配个性化的业务开发需求。这里我们可以总结下两种不同更新`state`的区别：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202506101630799.png" width=80%></div>

&emsp;&emsp;此外，除了`output_key`和`state_delta` 之外，还有一种方式可以实现状态的更新，那就是使用`CallbackContext` 和 `ToolContext`。

&emsp;&emsp;我们可以回忆一下，`output_key` 仅用于保存`Agent`生成响应后的数据，而`Agent`实际上承担着规划、生成自然语言响应、生成调用工具的解析参数等任务，那么如果我们希望在`Agent`执行过程中，根据`Agent`的执行情况来更新执行过程中所有关键的`State`（如工具调用指令），除了完全通过`state_delta` 来实现，`ADK`提供了一个中间层状态管理方案：`CallbackContext`和`ToolContext` ，用于在特定执行生命周期中提供**自动化的状态管理**。两者的执行时机和控制粒度介于`output_key`和`state_delta`之间，正如我们在本小节的最开始写到的三种`state`管理方式区别如下表所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>三种状态管理方式对比</font></p>
<div class="center">

| 方法 | 执行时机 | 控制粒度 | 适用场景 |
|------|----------|----------|----------|
| **`output_key`** | Agent 响应完成后 | 粗粒度 | 简单的响应保存 |
| **`state_delta`** | 任意时刻 | 最细粒度 | 复杂逻辑、系统事件 |
| **`ContextContext/ToolContext`** | 回调/工具执行期间 | 中等粒度 | 执行过程中的状态管理 |

&emsp;&emsp;这两种方法需要在工具调用或者回调函数的场景中才会触发，因此这里大家先做了解，后续在介绍`ADK`的回调函数和工具调用时，再展开详细的介绍。

# 二、ADK Agent 类型分类

&emsp;&emsp;在 `Google ADK` 中，`Agent` 主要分为三类：`LLM Agent`、`Workflow Agent`和`Custom Agent`。这三类 `Agent` 各司其职，相互配合可构建复杂的应用。图片来源：https://google.github.io/adk-docs/agents/

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202506111355625.png" width=60%></div>

- `LLM Agent`：利用大模型对自然语言进行理解、推理和生成，能够动态决策下一步动作或调用何种工具；
- `Workflow Agent`：包含`SequentialAgent`、`LoopAgent`和`ParallelAgent`三种模式，不依赖大模型进行决策，而是按照预定义的流程执行其子`Agent`，确保可预测、可控制的执行路径；
- `Custom Agent`：通过继承`BaseAgent` 并自定义其 `_run_async_impl` 执行逻辑，可以实现复杂的条件分支、外部系统调用或其他非标准流程。在内置的大模型/工作流模式无法满足需求时，可以选择编写自定义 `Agent`;

&emsp;&emsp;核心区别可总结为：`LLM Agent` 非确定性、依赖 LLM 进行推理；`Workflow Agent` 确定性、依赖预设逻辑控制流程；`Custom Agent` 则灵活性最高，可实现独特流程。下表对比了三者的核心能力：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>三种 Agent 类型对比</font></p>
<div class="center">

| Agent 类型       | 核心能力                      | 驱动方式          | 执行特点              | 应用场景                                           |
|------------------|-------------------------------|-------------------|-----------------------|----------------------------------------------------|
| **LLM Agent**    | 自然语言理解、推理与生成      | 大语言模型（LLM） | 非确定性，动态推理    | 对话生成、智能问答、工具调用、代码执行等         |
| **Workflow Agent** | 流程控制与任务编排            | 预定义逻辑结构    | 确定性，可预测         | 顺序处理（Sequential）、并行处理（Parallel）、迭代处理（Loop） |
| **Custom Agent** | 任意自定义逻辑与状态管理      | 自定义代码        | 灵活度最高，完全可控   | 复杂条件判断、动态流程、外部系统集成等             |


&emsp;&emsp;接下来我们就从`LLM Agent` 开始展开详细的介绍，同时也会结合特定的`Agent`实例重点讲解在`ADK`中关于工具调用、`MCP`接入，多代理协作等核心功能。

# 三、 LlmAgent 思考代理

&emsp;&emsp;我们在之前的演示中一直使用创建`Agent`的方法如下所示：

```python
    from google.adk.agents import Agent

    agent = Agent(
        name="chatbot",
        model=model,
        instruction="请查找用户指定国家的首都。你必须使用工具",
    )
```

&emsp;&emsp;其实这个`Agent`就是`LLM Agent`。我们可以从源码中明确的看到：https://github.com/google/adk-python/blob/main/src/google/adk/agents/llm_agent.py

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202506111417855.png" width=60%></div>

&emsp;&emsp;实际上，`Agent` 只是 `LlmAgent` 的一个别名。所以我们也可以直接使用 `LlmAgent` 来创建 `Agent`。如下代码所示：

In [56]:
import os
from google.adk.models.lite_llm import LiteLlm
from google.adk.runners import Runner
from google.adk.sessions import DatabaseSessionService
from google.genai import types 

from dotenv import load_dotenv
load_dotenv(override=True)


# 配置信息，注意这里不再预设置 SESSION_ID
DS_API_KEY = os.getenv("DS_API_KEY")
DS_BASE_URL = os.getenv("DS_BASE_URL")

APP_NAME = "llmagent_app"
USER_ID = "user_1"
SESSION_ID = "session_101"


# 数据库配置
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'adk',
    'user': 'postgres',
    'password': 'snowball2019'
}

# 生成数据库连接字符串
DATABASE_URL = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"

# 创建 SessionService 对象
session_service = DatabaseSessionService(DATABASE_URL)

# 创建会话
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
)

# 创建模型和 Agent
model = LiteLlm(
    model="deepseek/deepseek-chat",  
    api_base=DS_BASE_URL,
    api_key=DS_API_KEY
)

&emsp;&emsp;导入`LlmAgent` 类并使用`LlmAgent` 类创建`Agent` 对象实例：

In [57]:
from google.adk.agents import LlmAgent

# 创建 Agent 对象实例
agent = LlmAgent(
    name="chatbot",
    model=model,
    instruction="你是一个乐于助人的中文助手。",
)

&emsp;&emsp;使用`Runner`运行`Agent`，接收用户的输入并生成大模型的响应。

In [58]:
# 创建 Runner 对象实例
runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service
)


# 创建用户消息
query = "你好，请你介绍一下你自己"
content = types.Content(role='user', parts=[types.Part(text=query)])


# 运行
async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
    if event.content and event.content.parts and event.content.parts[0].text:
        print(event.content.parts[0].text)

你好！😊 我是你的智能助手，一个由人工智能驱动的对话伙伴。我的核心功能是为你提供信息、解答问题、协助完成各种任务（比如学习、写作、计算、翻译等），或者单纯陪你聊聊天。  

**关于我：**  
1️⃣ **知识覆盖广**：我的知识库涵盖科学、技术、文化、生活等多个领域（但截至2024年的新事件可能无法实时更新）。  
2️⃣ **多语言能力**：可以中英文切换，也支持其他语言的简单交流或翻译。  
3️⃣ **无主观意识**：我没有情感或个人观点，回答基于算法和数据，但会尽量模拟自然对话。  
4️⃣ **隐私保护**：对话内容默认不存储，请避免分享敏感信息哦。  

你可以随时告诉我你的需求，比如：  
• “帮我总结这篇文章”  
• “推荐周末放松的方法”  
• “用Python写一个爬虫代码”  

期待能帮到你！✨


&emsp;&emsp;这里可以看到，`LlmAgent`和`Agent`的应用方法是完全一样的，因为本质上`Agent`就是`LlmAgent`，只是别名而已。大家可以根据自己的喜好选择使用`Agent`或者`LlmAgent`。

&emsp;&emsp;对于`LlmAgent`类型，在`ADK`中主要承担的角色是依托大模型进行推理和生成，所以**它是非确定性的**，会根据用户输入的不同需求，会生成不同的响应。同时也因为它需要做不同的决策，所以我们就需要在创建`LlmAgent`时，赋予它对应的处理能力，核心就包括以下四点：

- **明确`LlmAgent`的角色和职责**：在创建 `LlmAgent` 时指定大模型、名称和描述，并明确它所能承担的任务要求和处理规范；
- **工具/MCP接入**：为`LlmAgent`装备多种工具/MCP，
- **多步规划**：它需要具备多步规划的能力，能够根据用户输入的不同需求，生成不同的响应；
- **数据结构约束**：约束输入和输出的数据格式，确保符合预期；

   .......

&emsp;&emsp;由此，在创建`LlmAgent`实例时，就有一系列不同的类型的参数来控制它的行为。完整支持接收的参数说明如下表所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>LlmAgent 参数说明</font></p>
<div class="center">

| 参数名                     | 类型                             | 默认值                  | 描述                                                                                     |
|--------------------------|--------------------------------|----------------------|----------------------------------------------------------------------------------------|
| `model`                  | `Union[str, BaseLlm]`         | `''`                 | 要使用的模型。                                    |
| `instruction`            | `Union[str, InstructionProvider]` | `''`                 | 指导代理行为的大模型模型指令。                                                               |
| `global_instruction`     | `Union[str, InstructionProvider]` | `''`                 | 整个代理树中所有代理的指令，仅在根代理中生效。                                               |
| `tools`                  | `list[ToolUnion]`             | `Field(default_factory=list)` | 该代理可用的工具。                                                                         |
| `generate_content_config` | `Optional[types.GenerateContentConfig]` | `None`               | 附加内容生成配置。                                                                         |
| `disallow_transfer_to_parent` | `bool`                       | `False`              | 禁止大模型转移控制权到父代理。                                                               |
| `disallow_transfer_to_peers`  | `bool`                       | `False`              | 禁止大模型转移控制权到同级代理。                                                             |
| `include_contents`       | `Literal['default', 'none']`   | `'default'`          | 是否在模型请求中包含内容。                                                                  |
| `input_schema`           | `Optional[type[BaseModel]]`    | `None`               | 当代理作为工具使用时的输入模式。                                                            |
| `output_schema`          | `Optional[type[BaseModel]]`    | `None`               | 代理回复时的输出模式。                                                                      |
| `output_key`             | `Optional[str]`                | `None`               | 存储代理输出的会话状态中的键。                                                              |
| `planner`                | `Optional[BasePlanner]`        | `None`               | 指示代理逐步制定计划并执行。                                                                |
| `code_executor`          | `Optional[BaseCodeExecutor]`   | `None`               | 允许代理使用提供的 CodeExecutor 执行模型响应中的代码块。                                     |
| `before_model_callback`   | `Optional[BeforeModelCallback]` | `None`               | 在调用 LLM 之前调用的回调或回调列表。                                                       |
| `after_model_callback`    | `Optional[AfterModelCallback]`  | `None`               | 在调用 LLM 之后调用的回调或回调列表。                                                       |
| `before_tool_callback`    | `Optional[BeforeToolCallback]`  | `None`               | 在调用工具之前调用的回调或回调列表。                                                       |
| `after_tool_callback`     | `Optional[AfterToolCallback]`   | `None`               | 在调用工具之后调用的回调或回调列表。                                                       |

&emsp;&emsp;其中最基础的四个参数作用范围如下所示:

- `model` （必填）： 指定支持此代理推理的底层大模型。
- `name` （必填）： 每个代理都需要一个唯一的字符串标识符。`name` 在多代理系统中的定义比较关键，代理间会通过`name`来进行任务执行参考或直接委派任务。
- `description` （可选，在多代理架构中比较关键）： 提供简洁的代理功能的摘要。此描述主要用于区别其他大模型代理，以确定是否应将任务分配给此代理。
- `instruction` （可选）： 提供一个简短的指令，描述代理应该如何执行任务。

&emsp;&emsp;`LlmAgent` 可支持接收的参数非常多，除了上述四个基础的参数外，各个不同类型的参数其实需要特定的开发场景才能发挥实际的作用。比如像`global_instruction`、`disallow_transfer_to_parent`、`disallow_transfer_to_peers`等参数往往作用于多代理系统中。因此各个参数的应用方法，我们需要结合具体的开发场景来具体分析。

&emsp;&emsp;接下来我们从`tools`参数开始，来介绍`LlmAgent`的工具接入规范。

# 四、`LlmAgent` 接入外部工具

&emsp;&emsp;`Google ADK` 框架下创建`Agent` 时，可以通过`tools` 参数来添加工具，这里与其他框架不同的是：在`ADK` 框架中，可以直接将一个普通的`Python`函数添加到`tools` 参数中，`ADK` 会自动将其封装为`FunctionTool`，并不需要通过类似`@tool` 的装饰器来进行转化。比如我们定义两个普通的函数用来进行快速的测试：

In [59]:
from datetime import datetime
import psutil
import platform

# 实际有用的工具函数
def get_current_datetime() -> str:
    """获取当前精确的日期和时间，包括毫秒。"""
    now = datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]

def get_system_info() -> str:
    """获取当前系统的详细信息，包括CPU、内存使用率等。"""
    try:
        cpu_percent = psutil.cpu_percent(interval=1)
        memory = psutil.virtual_memory()
        disk = psutil.disk_usage('/')
        
        info = f"""系统信息:
                - 操作系统: {platform.system()} {platform.release()}
                - CPU使用率: {cpu_percent}%
                - 内存使用率: {memory.percent}% ({memory.used // (1024**3)}GB / {memory.total // (1024**3)}GB)
                - 磁盘使用率: {disk.percent}% ({disk.used // (1024**3)}GB / {disk.total // (1024**3)}GB)
                - Python版本: {platform.python_version()}"""
        
        return info
    except Exception as e:
        return f"获取系统信息时出错: {str(e)}"


# 将函数直接添加到 Agent 的工具列表
capital_agent = LlmAgent(
    name="chatbot",
    model=model,
    instruction="你是一个多功能的助手，可以调用外部工具获取当前的日期和时间，以及系统的详细信息。",
    tools=[get_current_datetime, get_system_info]  # ADK 会自动封装此函数为 FunctionTool
)

&emsp;&emsp;接下来执行运行测试，看看效果如何。

In [60]:
# 创建会话
session = await session_service.get_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
)

# 创建 Runner 对象实例
runner = Runner(
    agent=capital_agent,
    app_name=APP_NAME,
    session_service=session_service
)

# 创建用户消息
query = "现在几点了？"
content = types.Content(role='user', parts=[types.Part(text=query)])

# 运行
async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
    print(f"当前事件来自: {event.author}")
    # 访问文本内容
    if event.content and event.content.parts:    
        # 如果事件中存在工具调用，则打印执行的工具和参数
        if event.get_function_calls():
            print(f"需要执行外部工具:")
            for call in event.get_function_calls():
                tool_name = call.name
                arguments = call.args 
                print(f"  执行的工具是: {tool_name}, 传递的参数是: {arguments}")


        # 如果存在工具调用且工具调用成功，则打印工具调用的结果
        if event.get_function_responses():
            print(f"工具调用成功:")
            for response in event.get_function_responses():
                print(f"  工具：{response.name} 调用的结果是: {response.response}")

        elif event.content.parts[0].text:
            print(f"最终响应: {event.content.parts[0].text}")

当前事件来自: chatbot
需要执行外部工具:
  执行的工具是: get_current_datetime, 传递的参数是: {}
当前事件来自: chatbot
工具调用成功:
  工具：get_current_datetime 调用的结果是: {'result': '2025-06-11 17:32:21.311'}
当前事件来自: chatbot
最终响应: 现在是2025年6月11日，下午5点32分21秒。 😊


&emsp;&emsp;通过`Event`事件类型的打印能够清晰的看到，`LlmAgent`在执行过程中是先执行的`get_current_datetime`函数获取到当前的时间才生成最终的响应。同样可以进行另一个工具的测试：

In [61]:
# 创建会话
session = await session_service.get_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
)

# 创建 Runner 对象实例
runner = Runner(
    agent=capital_agent,
    app_name=APP_NAME,
    session_service=session_service
)

# 创建用户消息
query = "帮我获取一下系统信息"
content = types.Content(role='user', parts=[types.Part(text=query)])

# 运行
async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
    print(f"当前事件来自: {event.author}")
    # 访问文本内容
    if event.content and event.content.parts:    
        # 如果事件中存在工具调用，则打印执行的工具和参数
        if event.get_function_calls():
            print(f"需要执行外部工具:")
            for call in event.get_function_calls():
                tool_name = call.name
                arguments = call.args 
                print(f"  执行的工具是: {tool_name}, 传递的参数是: {arguments}")

        # 如果存在工具调用且工具调用成功，则打印工具调用的结果
        if event.get_function_responses():
            print(f"工具调用成功:")
            for response in event.get_function_responses():
                print(f"  工具：{response.name} 调用的结果是: {response.response}")

        elif event.content.parts[0].text:
            print(f"最终响应: {event.content.parts[0].text}")

当前事件来自: chatbot
需要执行外部工具:
  执行的工具是: get_system_info, 传递的参数是: {}
当前事件来自: chatbot
工具调用成功:
  工具：get_system_info 调用的结果是: {'result': '系统信息:\n                - 操作系统: Windows 11\n                - CPU使用率: 4.2%\n                - 内存使用率: 87.9% (13GB / 15GB)\n                - 磁盘使用率: 23.6% (78GB / 331GB)\n                - Python版本: 3.12.0'}
当前事件来自: chatbot
最终响应: 以下是当前的系统信息：  

- **操作系统**：Windows 11  
- **CPU使用率**：4.2%  
- **内存使用率**：87.9% (13GB / 15GB)  
- **磁盘使用率**：23.6% (78GB / 331GB)  
- **Python版本**：3.12.0  

如果需要进一步分析或帮助，请告诉我！ 😊


&emsp;&emsp;在`Google ADK`中，`LlmAgent` 的工具接入规范如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202506101839153.png" width=80%></div>

&emsp;&emsp;这个过程并不难理解，只是大模型调用外部工具的标准流程，而仅仅是`ADK`做了自动的转化而已。但实际上，结合着`ADK`底层实现的逻辑，我们可以总结出在`ADK`中接入工具的一些最佳实践规范，用来提升`Agent`调用外部工具的效率及准确性。其完整的底层函数调用逻辑如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202506111746018.png" width=60%></div>


&emsp;&emsp;对`ADK`中工具函数的转化细节感兴趣的同学可以参考上图中的调用关系结合源码进行详细的了解。我们这里直接来看一下工具接入的最佳实践策略。

In [62]:
capital_agent.tools

[<function __main__.get_current_datetime() -> str>,
 <function __main__.get_system_info() -> str>]

&emsp;&emsp;这里我们再添加一个有实际入参的工具函数。

In [63]:
import secrets
import string

def generate_secure_password(length: int = 12, include_symbols: bool = True) -> str:
    """生成指定长度的安全随机密码。
    
    Args:
        length: 密码长度，默认12位
        include_symbols: 是否包含特殊符号，默认True
    """
    if length < 4 or length > 50:
        return "密码长度必须在4-50之间"
    
    chars = string.ascii_letters + string.digits
    if include_symbols:
        chars += "!@#$%^&*"
    
    return ''.join(secrets.choice(chars) for _ in range(length))


# 将函数直接添加到 Agent 的工具列表
capital_agent = LlmAgent(
    name="chatbot",
    model=model,
    instruction="你是一个多功能的助手，可以调用外部工具获取当前的日期和时间，以及系统的详细信息，同时也可以生成安全的随机密码用。",
    tools=[generate_secure_password, get_current_datetime, get_system_info]  # ADK 会自动封装此函数为 FunctionTool
)

&emsp;&emsp;接下来，我们尝试在`LlmAgent`的对象结构中，提取出由`Google ADK`自动根据工具函数生成的`Json Schema`表示。代码如下：

In [64]:
import json

# canonical_tools 函数会返回一个列表，列表中每个元素是一个 FunctionTool 对象
canonical_tools = await capital_agent.canonical_tools()

for i, tool in enumerate(canonical_tools, 1):
    print(f"\n{'='*70}")
    print(f"🔍 工具 {i}: {tool.name}")
    print(f"{'='*70}")
    
    # 获取ADK生成的FunctionDeclaration
    func_decl = tool._get_declaration()
    
    print(f"📋 ADK生成的FunctionDeclaration对象:")
    print(f"  类型: {type(func_decl)}")
    
    # # 直接打印对象的内容
    # print(f"\n🔧 对象属性:")
    # print(f"  - dir(func_decl): {[attr for attr in dir(func_decl) if not attr.startswith('_')]}")
    
    print(f"\n📤 ADK生成的JSON Schema :")
    
    # to_json_dict()
    print(f"\n✅ to_json_dict() - 转换为JSON字典:")
    try:
        json_dict_result = func_decl.to_json_dict()
        print(json.dumps(json_dict_result, indent=2, ensure_ascii=False))
    except Exception as e:
        print(f"  错误: {e}")

Default value is not supported in function declaration schema for Google AI.
Default value is not supported in function declaration schema for Google AI.



🔍 工具 1: generate_secure_password
📋 ADK生成的FunctionDeclaration对象:
  类型: <class 'google.genai.types.FunctionDeclaration'>

📤 ADK生成的JSON Schema :

✅ to_json_dict() - 转换为JSON字典:
{
  "description": "生成指定长度的安全随机密码。\n\n    Args:\n        length: 密码长度，默认12位\n        include_symbols: 是否包含特殊符号，默认True\n    ",
  "name": "generate_secure_password",
  "parameters": {
    "properties": {
      "length": {
        "type": "INTEGER"
      },
      "include_symbols": {
        "type": "BOOLEAN"
      }
    },
    "required": [
      "length",
      "include_symbols"
    ],
    "type": "OBJECT"
  }
}

🔍 工具 2: get_current_datetime
📋 ADK生成的FunctionDeclaration对象:
  类型: <class 'google.genai.types.FunctionDeclaration'>

📤 ADK生成的JSON Schema :

✅ to_json_dict() - 转换为JSON字典:
{
  "description": "获取当前精确的日期和时间，包括毫秒。",
  "name": "get_current_datetime"
}

🔍 工具 3: get_system_info
📋 ADK生成的FunctionDeclaration对象:
  类型: <class 'google.genai.types.FunctionDeclaration'>

📤 ADK生成的JSON Schema :

✅ to_json_dict() - 转换为JSON字典:


&emsp;&emsp;通过这个机制能清楚，在`ADK`中自动解析外部工具时关注的核心是：外部函数的名称、函数注释和函数入参的类型。因此一个良好的外部工具函数在定义时需要遵循以下三个原则：

- 原则一：定义工具函数的参数时，其一<font color="red">**使用标准的`Json 可序列化（serializable）`类型</font>。其二<font color="red">避免给参数设置默认值</font>，因为大模型无法解析参数的默认值。其三<font color="red">**参数量越少越好**</font>。最后<font color="red">**函数工具名称要有实际的意义**</font>，定义能够清晰反映函数用途及其输入含义的名称。**

&emsp;&emsp;`JSON 可序列化（serializable）`类型，指的是在`Python`中，只要一个值可以直接被 `json.dumps()` 无损编码，就称为 `JSON 可序列化`。常见映射如下：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>JSON 可序列化类型</font></p>
<div class="center">

| JSON 原语   | Python 映射       | 说明                         |
| --------- | --------------- | -------------------------- |
| `string`  | `str`           | 任何字符序列                     |
| `number`  | `int`, `float`  | 所有整数与浮点数                   |
| `boolean` | `bool`          | `True` / `False`           |
| `null`    | `None`          | 空值                         |
| `array`   | `list`, `tuple` | 元素本身也必须可序列化                |
| `object`  | `dict`          | key 必须是 `str`，value 必须可序列化 |


&emsp;&emsp;因此对以下两种写法：

```python
    # 写法 A：有类型注解 + 默认值
    def generate_secure_password(length: int, include_symbols: bool = True) -> str: ...
    # 写法 B：无类型注解 + 无默认值
    def generate_secure_password(length, include_symbols): ...
```

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>写法对比</font></p>
<div class="center">

| 对比项      | 写法 A                               | 写法 B                                                |
| -------- | ---------------------------------- | --------------------------------------------------- |
| **类型注解** | ✅ 推荐（框架能生成精确的 `JSON‑Schema`）         | ❌ 不推荐（框架只能把它们当作 `string` 或 `any`，大模型获得的 `schema` 含糊） |
| **默认值**  | ⚠️ 不推荐（违背“避免默认值”最佳实践）              | ✅ 没默认值                                        
| **整体评价** | → **改进**：去掉默认值或改成 `Optional[bool]` | → **改进**：补全类型注解                                     |


&emsp;&emsp;因此，建议的标准写法是：

```python
    def generate_secure_password(
        length: int,
        include_symbols: bool,
    ) -> Dict[str, Any]:
```

- **原则二：<font color="red">**函数工具的首选返回类型是`字典`类型，所以建议使用键值对来构造响应**</font>。同时使返回值尽可能具有描述性。最佳做法是，在返回字典中包含一个`status`键，以指示整体结果（例如，`success`、`error`、`pending`），从而向大模型提供有关操作状态的清晰信号。默认情况下，`ADK`框架会自动将其包装到一个`result` 的字典中。**

&emsp;&emsp;如果定义的外部工具函数返回的不是字典类型，框架会自动将其包装到一个名为 “result” 的字典中。形式如下：

```python
    def generate_secure_password(length: int, include_symbols: bool) -> Dict[str, Any]:
        """生成指定长度的安全随机密码。
        
        Args:
            length: 密码长度，默认12位
            include_symbols: 是否包含特殊符号，默认True
        """
        if length < 4 or length > 50:
            return "密码长度必须在4-50之间"
        
        chars = string.ascii_letters + string.digits
        if include_symbols:
            chars += "!@#$%^&*"
        
        return ''.join(secrets.choice(chars) for _ in range(length))

    # ADK 会自动转换成：
    { "result": "''.join(secrets.choice(chars) for _ in range(length))" }
```

&emsp;&emsp;所以建议尽可能通过键值对来构造响应，如下所示：

```python
    def generate_secure_password(length: int, include_symbols: bool) -> Dict[str, str]:
        if not 4 <= length <= 50:
            return {
                "status": "error",
                "error_message": "密码长度必须在 4‑50 之间"
            }

        chars = string.ascii_letters + string.digits
        if include_symbols:
            chars += "!@#$%^&*"

        password = ''.join(secrets.choice(chars) for _ in range(length))
       
        # 返回字典类型
        return {
            "status": "success",
            "password": password
        }
```

- **原则三：<font color="red">**清晰的编写函数注释，包含解释函数用途、参数含义以及预期的返回值三部分。**</font>**

```python

    def generate_secure_password(length: int, include_symbols: bool) -> Dict[str, Any]:
        """
        生成指定长度的安全随机密码。

        Args:
            length: 密码长度（必填，4‑50 之间）
            include_symbols: 是否包含特殊符号（必填）

        Returns:
            {
                "status": "success" | "error",
                "password": "<随机密码>",          # status 为 success 时存在
                "error_message": "<错误原因>"      # status 为 error 时存在
            }
        """
        if not 4 <= length <= 50:
            return {
                "status": "error",
                "error_message": "密码长度必须在 4‑50 之间"
            }

        chars = string.ascii_letters + string.digits
        if include_symbols:
            chars += "!@#$%^&*"

        password = ''.join(secrets.choice(chars) for _ in range(length))
        return {
            "status": "success",
            "password": password
        }
```

&emsp;&emsp;因此，其实我们上面的三个工具函数都是不合格的。我们可以进行如下调整和优化。

In [65]:
from __future__ import annotations

from datetime import datetime
from typing import Dict, Any

import platform
import secrets
import string

import psutil  # pip install psutil


# ----------------------------------------------------------------------
# 工具 1：获取当前日期时间
# ----------------------------------------------------------------------
def get_current_datetime() -> Dict[str, str]:
    """
    获取当前精确日期时间（到毫秒）。

    Returns:
        {
            "status": "success" | "error",
            "datetime": "YYYY‑MM‑DD HH:MM:SS.mmm",   # status=success 时存在
            "error_message": "<错误信息>"               # status=error  时存在
        }
    """
    try:
        now = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
        return {"status": "success", "datetime": now}
    except Exception as exc:
        return {"status": "error", "error_message": str(exc)}


# ----------------------------------------------------------------------
# 工具 2：获取系统资源信息
# ----------------------------------------------------------------------
def get_system_info() -> Dict[str, Any]:
    """
    获取 CPU、内存、磁盘 等系统资源使用情况。

    Returns:
        {
            "status": "success" | "error",
            "os":               "<操作系统>",
            "cpu_percent":      32.5,
            "memory_percent":   71.2,
            "memory_used_gb":   8,
            "memory_total_gb":  16,
            "disk_percent":     55.6,
            "disk_used_gb":     120,
            "disk_total_gb":    240,
            "python_version":   "3.12.3",
            "error_message":    "<错误信息>"   # status=error 时存在
        }
    """
    try:
        cpu_percent = psutil.cpu_percent(interval=1)
        memory = psutil.virtual_memory()
        disk = psutil.disk_usage("/")

        return {
            "status":         "success",
            "os":             f"{platform.system()} {platform.release()}",
            "cpu_percent":    cpu_percent,
            "memory_percent": memory.percent,
            "memory_used_gb": memory.used // (1024**3),
            "memory_total_gb": memory.total // (1024**3),
            "disk_percent":   disk.percent,
            "disk_used_gb":   disk.used // (1024**3),
            "disk_total_gb":  disk.total // (1024**3),
            "python_version": platform.python_version(),
        }
    except Exception as exc:
        return {"status": "error", "error_message": str(exc)}


# ----------------------------------------------------------------------
# 工具 3：生成安全随机密码
# ----------------------------------------------------------------------
def generate_secure_password(
    length: int,
    include_symbols: bool
) -> Dict[str, str]:
    """
    生成指定长度的安全随机密码。

    Args:
        length (int): 密码长度（4–50）。
        include_symbols (bool): 是否包含特殊符号。

    Returns:
        {
            "status": "success" | "error",
            "password":       "<随机密码>",     # status=success 时存在
            "error_message":  "<错误信息>"      # status=error  时存在
        }
    """
    if not 4 <= length <= 50:
        return {
            "status": "error",
            "error_message": "密码长度必须在 4–50 之间"
        }

    chars = string.ascii_letters + string.digits
    if include_symbols:
        chars += "!@#$%^&*"

    password = "".join(secrets.choice(chars) for _ in range(length))
    return {"status": "success", "password": password}


&emsp;&emsp;建议大家在后续的开发中，如果涉及到自定义外部函数的接入严格遵循上述的定义原则，以提升在`ADK`框架中应用外部工具的准确性。

&emsp;&emsp;以上就是`Google ADK`框架在接入自定义工具所需要掌握的全部内容，而关于更多的内置工具函数的使用，`CrewAI`和`Langchain`工具的集成，以及`MCP Server`的接入，我们在下一小节课程中继续进行介绍。